### Get Packages 

In [55]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
import datetime

%run ml_helpers.ipynb
%run ml.ipynb

         Unnamed: 0  Beat  Year  Month   Watch  PRCP  SNOW  TMAX  TMIN  \
1145280     1145280   812  2019      4  Second  0.00   0.0    66    50   
1239002     1239002   725  2019      8  Second  0.00   0.0    84    65   
1295971     1295971   213  2019     11   First  0.00   0.0    33    17   
1134302     1134302  2432  2019      4   Third  0.16   0.0    70    53   
1218458     1218458  1131  2019      7   Third  0.00   0.0    86    68   
...             ...   ...   ...    ...     ...   ...   ...   ...   ...   
1237656     1237656  1832  2019      8   Third  0.04   0.0    85    71   
1204555     1204555   124  2019      7  Second  0.00   0.0    89    67   
1245917     1245917   924  2019      8   Third  0.00   0.0    74    62   
1127254     1127254   834  2019      3  Second  0.00   0.0    65    45   
1235591     1235591   933  2019      8   First  0.04   0.0    78    69   

         count_l_stops  ...  count_restaurants  count_bars  count_daycares  \
1145280            0.0  ...      

Finished standardizing...
Working on: [2015]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [116]:
#Get Final data: 
data = pd.read_csv("../intermediate_data/high_crime_labeled.csv")
data = data.drop(columns=["beat", "beat_num", "Crimes", "Arrest"])

data = convert_to_categorical(data, ["district", "sector",
                                    "Month",
                                     "Watch", "Beat"])
data.drop(columns = ["Unnamed: 0"], inplace=True)
data_list_20 = prep_data_test(data, "high_crime",
                                        2, "Year", ["district", "sector", 
                                                    "Month", "Beat", "Watch"])


      district sector  Year Month   Watch  Beat  Serious  Domestic       TMAX  \
29557       17      1  2018     1   First  1713      1.0         2  33.201426   
29558       17      1  2018     1  Second  1713      8.0         4  33.201426   
29559       17      1  2018     1   Third  1713     11.0         4  33.201426   
29560       17      1  2018     2   Third  1713      5.0         3  39.791635   
29561       17      1  2018     2   First  1713      6.0         2  39.791635   
...        ...    ...   ...   ...     ...   ...      ...       ...        ...   
49256        3      1  2019    11  Second   312     13.0        11  43.761013   
49257        3      1  2019    11   Third   312     13.0         8  43.761013   
49258        3      1  2019    12  Second   312     20.0         8  42.647269   
49259        3      1  2019    12   First   312      7.0         8  42.647269   
49260        3      1  2019    12   Third   312     13.0         4  42.647269   

            TMIN  ...  coun

Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2017 2018]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2016 2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [117]:
data_list_20[0][0]

,Year,Serious,Domestic,TMAX,TMIN,PRCP,SNOW,high_crime,count_l_stops,count_bus_stops,...,Beat_2524,Beat_2525,Beat_2531,Beat_2532,Beat_2533,Beat_2534,Beat_2535,Watch_First,Watch_Second,Watch_Third
29557,2018,-1.254018,-0.644941,-1.376112,-1.389057,-1.299674,0.231712,0.0,3.214337,-0.589002,...,0,0,0,0,0,0,0,1,0,0
29558,2018,-0.620498,-0.112728,-1.376112,-1.389057,-1.299674,0.231712,0.0,3.214337,-0.589002,...,0,0,0,0,0,0,0,0,1,0
29559,2018,-0.348990,-0.112728,-1.376112,-1.389057,-1.299674,0.231712,0.0,3.214337,-0.589002,...,0,0,0,0,0,0,0,0,0,1
29560,2018,-0.892007,-0.378835,-1.034854,-1.191190,0.608665,3.009861,0.0,3.214337,-0.589002,...,0,0,0,0,0,0,0,0,0,1
29561,2018,-0.801504,-0.644941,-1.034854,-1.191190,0.608665,3.009861,0.0,3.214337,-0.589002,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49256,2019,-0.167984,1.750018,-0.829309,-0.790072,-1.045229,-0.115556,0.0,1.952276,-0.250446,...,0,0,0,0,0,0,0,0,1,0
49257,2019,-0.167984,0.951698,-0.829309,-0.790072,-1.045229,-0.115556,0.0,1.952276,-0.250446,...,0,0,0,0,0,0,0,0,0,1
49258,2019,0.465536,0.951698,-0.886981,-0.889857,-1.225095,-0.297459,1.0,1.952276,-0.250446,...,0,0,0,0,0,0,0,0,1,0
49259,2019,-0.711001,0.951698,-0.886981,-0.889857,-1.225095,-0.297459,0.0,1.952276,-0.250446,...,0,0,0,0,0,0,0,1,0,0


In [118]:
def compute_scores_final_data(grid, model, model_name, data_list, target, target_year):
    '''
    Function that computes dataframe for results of best models
    Inputs:
        grid (dict): maps the model_name to a list which contains a dictionary mapping parameter (keys) to their 
                     values in the gest model
        model (sklearn): An sklearn algorithm
        model_name: a string assigned to the model, typically the same as model without the last parentheses
        data_list: list of tuples of dataframes. IN THIS CASE, this should include for 2020
        target (string): the column we seek to predict in the data frames
        target_year (int): 2020, the year we seek to predict
    Output:
        results (df): a dataframe with the results
    
    Note: the code that removes the params columns comes from here: https://stackoverflow.com/questions/19071199/drop-columns-whose-name-contains-a-specific-string-from-pandas-dataframe
    '''
    results, model, X_test_20 = results_by_year(model, 
                                    model_name, grid, 
                                    data_list, 
                                    target, 
                                    target_year)
    results["model_type"]= model_name
    results["params"]=results["parameters_2020"]
    
    bool_array = results.columns.str.startswith("parameters")
    results2 = results.loc[:,~bool_array]
    return results2, model, X_test_20

## Logistic Regression

In [119]:
#Logistic Regression!
grid_log_reg ={"LogisticRegression": [{'penalty': 'l2',
                              'C': 0.001,
                              'random_state': 0}]}

In [120]:
results_log, model, X_test = compute_scores_final_data(grid_log_reg, LogisticRegression(),
                                    "LogisticRegression", data_list_20, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]
precision is: (0.990185387131952,) |recall is: (0.37598343685300206,)
precision is: (0.987597911227154,) |recall is: (0.6011124354390147,)
precision is: (0.9846968805179518,) |recall is: (0.6765062676910635,)
precision is: (0.9649581589958159,) |recall is: (0.7564575645756457,)


In [149]:
results_log

,precision_2020,recall_2020,precision_2019,recall_2019,precision_2018,recall_2018,precision_2017,recall_2017,model_type,params
0,0.990185,0.375983,0.987598,0.601112,0.984697,0.676506,0.964958,0.756458,LogisticRegression,"{'penalty': 'l2', 'C': 0.001, 'random_state': 0}"


In [121]:
#Below code to make model coefficient df from Stack Exchange: https://stackoverflow.com/questions/26951880/scikit-learn-linear-regression-how-to-get-coefficients-respective-features
model_coefs = pd.concat([pd.DataFrame(X_test.columns), 
                         pd.DataFrame(np.transpose(model.coef_))], axis = 1)
model_coefs.columns = ["Predictor", "Coefficient"]
model_coefs["Coefficient"]=model_coefs["Coefficient"].abs()
model_coefs.sort_values(by="Coefficient", ascending=False, inplace=True)
model_coefs.head(10)

,Predictor,Coefficient
1,Serious,1.552341
332,Watch_First,0.362579
19,high_crime_geog_lag,0.294331
2,Domestic,0.257853
334,Watch_Third,0.182501
333,Watch_Second,0.180108
6,SNOW,0.099002
8,count_bus_stops,0.098973
4,TMIN,0.076224
13,count_entertainment,0.071749


## KNN

In [146]:
#KNN!
grid_knn = {"KNeighborsClassifier": [{'weights': 'uniform', 'n_neighbors': 20}]}
results_knn = compute_scores_final_data(grid_knn, KNeighborsClassifier(),
                                    "KNeighborsClassifier", data_list_20, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]
precision is: (0.8411811652035116,) |recall is: (0.43643892339544516,)
precision is: (0.8785399622404028,) |recall is: (0.5546285260230434,)
precision is: (0.896551724137931,) |recall is: (0.5677315002021835,)
precision is: (0.8852459016393442,) |recall is: (0.6199261992619927,)


In [148]:
results_knn[0]

,precision_2020,recall_2020,precision_2019,recall_2019,precision_2018,recall_2018,precision_2017,recall_2017,model_type,params
0,0.841181,0.436439,0.87854,0.554629,0.896552,0.567732,0.885246,0.619926,KNeighborsClassifier,"{'weights': 'uniform', 'n_neighbors': 20}"


## SVM: 

In [ ]:
grid_svm = { :[{}]}
results_svm = compute_scores_final_data()

## Naive Bayes: 

In [ ]:
grid_nb = { :[{}]}
results_nb = compute_scores_final_data()

## Decision Tree:

In [ ]:
grid_dt = { :[{}]}
results_dt = compute_scores_final_data()

## Random Forest:

In [ ]:
grid_rf = { :[{}]}
results_rf = compute_scores_final_data()

## Aggregate Files:

In [151]:
results = pd.concat([results_log, results_knn[0]
#                    , results_svm, results_nb, results_dt, results_rf
                    ])
results

,precision_2020,recall_2020,precision_2019,recall_2019,precision_2018,recall_2018,precision_2017,recall_2017,model_type,params
0,0.990185,0.375983,0.987598,0.601112,0.984697,0.676506,0.964958,0.756458,LogisticRegression,"{'penalty': 'l2', 'C': 0.001, 'random_state': 0}"
0,0.841181,0.436439,0.878540,0.554629,0.896552,0.567732,0.885246,0.619926,KNeighborsClassifier,"{'weights': 'uniform', 'n_neighbors': 20}"
